In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import keras
import os
import tensorboard
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorboard.plugins.hparams import api as hp

from photoz_utils import *
from DataMaker import *

In [2]:
IMAGE_SHAPE = (5, 127, 127)
GB_LIMIT = 10

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(GB_LIMIT*1000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
NUM_DENSE_UNITS = 200
BATCH_SIZE = 256
NUM_EPOCHS = 1000
LEARNING_RATE = 0.0001
Z_MAX = 4
hparams = {
    'num_dense_units': NUM_DENSE_UNITS,
    'batch_size': BATCH_SIZE,
    'num_epochs': NUM_EPOCHS,
    'learning_rate': LEARNING_RATE,
    'z_max': Z_MAX
}

In [5]:
TRAIN_PATH = f'/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected.hdf5'
VAL_PATH = f'/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation.hdf5'
TEST_PATH = f'/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing.hdf5'

In [ ]:
column_names = ['object_id', 'specz_redshift', 'g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag']
with h5py.File(TRAIN_PATH, 'r') as hf:
    train_df = pd.DataFrame()
    for name in column_names:
        train_df[name] = hf[name][:]
    train_images = np.asarray(hf['image'][:])
    
with h5py.File(VAL_PATH, 'r') as hf:
    val_df = pd.DataFrame()
    for name in column_names:
        val_df[name] = hf[name][:]
    val_images = np.asarray(hf['image'][:])
    
with h5py.File(TEST_PATH, 'r') as hf:
    test_df = pd.DataFrame()
    for name in column_names:
        test_df[name] = hf[name][:]
    test_images = np.asarray(hf['image'][:])

In [ ]:
mags = ['g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag']
X_train, X_val, X_test = train_df[mags], val_df[mags], test_df[mags]
y_train, y_val, y_test = train_df['specz_redshift'], val_df['specz_redshift'], test_df['specz_redshift']
OID_train, OID_val, OID_test = train_df['object_id'], val_df['object_id'], test_df['object_id']

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [ ]:
# for now, only run the NN architecture
# add callbacks

In [ ]:
import keras.backend as K

def calculate_loss(z_photo, z_spec):
    """
    HSC METRIC. Returns an array. Loss is accuracy metric defined by HSC, meant
    to capture the effects of bias, scatter, and outlier all in one. This has
    uses for both point and density estimation.
    z_photo: array
        Photometric or predicted redshifts.
    z_spec: array
        Spectroscopic or actual redshifts.
    """
    dz = delz(z_photo, z_spec)
    gamma = 0.15
    denominator = 1.0 + K.square(dz/gamma)
    loss = 1 - 1.0 / denominator
    return loss

In [ ]:
input_nn = Input(shape=X_train.shape[1:])
hidden1 = Dense(hparams['num_dense_units'], activation="relu")(input_nn)
hidden2 = Dense(hparams['num_dense_units'], activation="relu")(hidden1)
hidden3 = Dense(hparams['num_dense_units'], activation="relu")(hidden2)
hidden4 = Dense(hparams['num_dense_units'], activation="relu")(hidden3)

input_cnn = Input(shape=(5,127,127))
conv1 = Conv2D(32, kernel_size=(3, 3), activation='tanh', padding='same', data_format='channels_first')(input_cnn)
pool1 = MaxPooling2D(pool_size = (2,2), data_format='channels_first')(conv1)
conv2 = Conv2D(64, kernel_size=(2,2), activation='tanh', padding='same', data_format='channels_first')(pool1)
pool2 = MaxPooling2D(pool_size = (2,2), data_format='channels_first')(conv2)
conv3 = Conv2D(128, kernel_size=(2,2), activation='tanh', padding='same', data_format='channels_first')(pool2)
pool3 = MaxPooling2D(pool_size = (2,2), data_format='channels_first')(conv3)
conv4 = Conv2D(256, kernel_size=(2,2), activation='tanh', padding='same', data_format='channels_first')(pool3)
pool4 = MaxPooling2D(pool_size = (2,2), data_format='channels_first')(conv4)
conv5 = Conv2D(256, kernel_size=(2,2), activation='tanh', padding='same', data_format='channels_first')(pool4)
pool5 = MaxPooling2D(pool_size = (2,2), data_format='channels_first')(conv5)
conv6 = Conv2D(512, kernel_size=(3,3),activation='relu', padding='same', data_format='channels_first')(pool5)
conv7 = Conv2D(512, kernel_size=(2,2),activation='relu', padding='same', data_format='channels_first')(conv6)
flatten = Flatten()(conv7)
dense1 = Dense(512, activation='tanh')(flatten)
dense2 = Dense(128, activation='tanh')(dense1)
dense3 = Dense(32, activation='tanh')(dense2)

concat = Concatenate()([hidden4, dense3])
distribution_params = Dense(units=2)(concat)
output = tfp.layers.IndependentNormal(1)(distribution_params)
model = Model(inputs=[input_nn, input_cnn], outputs=[output])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=hparams['learning_rate']), loss=calculate_loss, metrics='mse')

In [ ]:
model_name = 'HSC_v6_NN_neurips_combined_with_5pool_v3'

checkpoint_filepath = os.path.join('/models/', model_name)+'/'+model_name
log_dir = os.path.join('/logs/', model_name)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

hparam_callback = hp.KerasCallback(log_dir, hparams)

In [ ]:
model.fit(x=[X_train, train_images], y=y_train, batch_size=hparams['batch_size'], epochs=hparams['num_epochs'], shuffle=True, verbose=1, validation_data=([X_val, val_images], y_val), callbacks=[tensorboard_callback, model_checkpoint_callback, hparam_callback])

In [ ]:
evaluated_model = model([X_test, test_images])

In [ ]:
# photoz = []
# for i in range(0,len(X_test)):
#     pred = model([X_test[i], test_images[i]])
#     photoz.append(pred)

In [ ]:
mean_arr = evaluated_model.mean().numpy()
std_arr = evaluated_model.stddev().numpy()

In [ ]:
y_pred = np.ravel(mean_arr) # to make ndim = 1
y_pred

In [ ]:
plot_predictions(y_pred, y_test)

In [ ]:
metrics = get_point_metrics(pd.Series(y_pred), pd.Series(y_test))

In [ ]:
metrics

In [ ]:
df = pd.DataFrame(y_pred, columns=['photoz'])
df['photz_err'] = np.ravel(std_arr)
df['specz'] = y_test
df['object_id'] = OID_test
if os.path.exists(f'/predictions/{model_name}') == False:
    os.makedirs(f'/predictions/{model_name}')
    
df.to_csv(f'/predictions/{model_name}/testing_predictions.csv', index=False)

In [ ]:
if os.path.exists(f'predictions/{model_name}') ==  False:
    os.makedirs(f'predictions/{model_name}')
    
metrics.to_csv(f'/predictions/{model_name}/testing_metrics.csv', index=False)

In [ ]:
y_train

In [ ]:
# v1 batch_size = 256
# v2 batch_size = None